In [1]:
# %pip install salabim
import salabim as sim

# The M/M/k/N queue

We consider an M/M/5/10 queue with Poisson arrivals, exponentially distributed service times,
$k=5$ servers, and $N-k = 5$ additional buffer spaces.

In [2]:
# Parameters
lambd = 4.0 # Arrival rate
mu = 1.0 # Service rate
capacity = 4
buffer = 4

env = sim.Environment()
servers = sim.Resource('servers', capacity=capacity)

# Inter-arrival and service time distributions
inter_arr_dist = sim.Exponential(rate=lambd)
service_dist = sim.Exponential(rate=mu)

# Counters
env.nArr = 0
env.nServed = 0
env.nBlocked = 0
env.nDelayed = 0

class Request(sim.Component):
    def process(self):
        env.nArr += 1
        if env.nArr % 100_000 == 0:
            print(env.nArr, env.nBlocked, env.nBlocked / env.nArr, len(servers.requesters()))

        # Admit to system if there is an available server or buffer space
        if servers.available_quantity() > 0 or servers.requesters().length() < buffer:
            if servers.available_quantity() == 0:
                env.nDelayed += 1
            yield self.request(servers)
            yield self.hold(service_dist.sample())
            env.nServed += 1
        else:
            env.nBlocked += 1

sim.ComponentGenerator(Request, iat=inter_arr_dist)
env.run(till=3e5)

print()
print('# Arrivals:', env.nArr)
print('Blocking probability:', env.nBlocked / env.nArr)
print('Delay probability:', env.nDelayed / env.nArr)
print()
print('Mean server occupancy:', servers.occupancy.mean()) # busy servers / capacity
print('Mean buffer length:', servers.requesters().length.mean())
print('Mean time in buffer:', servers.requesters().length_of_stay.mean())

100000 14130 0.1413 0
200000 28076 0.14038 0
300000 41616 0.13872 0
400000 55212 0.13803 3
500000 69093 0.138186 1
600000 83055 0.138425 1
700000 96719 0.13817 2
800000 110034 0.1375425 4
900000 123650 0.1373888888888889 0
1000000 137796 0.137796 0
1100000 152046 0.13822363636363635 0
1200000 165690 0.138075 0

# Arrivals: 1201530
Blocking probability: 0.13806896207335648
Delay probability: 0.5540336071508827

Mean server occupancy: 0.861315364545513
Mean buffer length: 1.382593343734158
Mean time in buffer: 0.40050558605557673


To compute theoretical values, refer to Section 12.4 of [this book](https://arxiv.org/pdf/1307.2968v25.pdf#page=200) by Prof. Moshe Zukerman.